In [1]:
## Necessary for Colab, not necessary for course environment
%pip install -q langchain langchain-nvidia-ai-endpoints gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.

In [2]:
import os
os.environ["NVIDIA_API_KEY"] = "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"

In [3]:
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [5]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
for model in ChatNVIDIA.get_available_models():
  print(model.id)

01-ai/yi-large
microsoft/phi-3-medium-4k-instruct
microsoft/kosmos-2
google/deplot
aisingapore/sea-lion-7b-instruct
google/gemma-2-2b-it
microsoft/phi-3-small-128k-instruct
google/paligemma
deepseek-ai/deepseek-coder-6.7b-instruct
meta/llama3-8b-instruct
meta/llama-3.1-405b-instruct
nvidia/neva-22b
nvidia/llama3-chatqa-1.5-8b
meta/llama3-70b-instruct
ibm/granite-34b-code-instruct
ibm/granite-8b-code-instruct
nvidia/llama3-chatqa-1.5-70b
google/gemma-2b
google/gemma-7b
mistralai/codestral-22b-instruct-v0.1
microsoft/phi-3-medium-128k-instruct
liuhaotian/llava-v1.6-mistral-7b
microsoft/phi-3-vision-128k-instruct
writer/palmyra-fin-70b-32k
upstage/solar-10.7b-instruct
writer/palmyra-med-70b
meta/llama-3.1-8b-instruct
microsoft/phi-3-mini-4k-instruct
google/gemma-2-27b-it
mistralai/mistral-7b-instruct-v0.3
liuhaotian/llava-v1.6-34b
google/codegemma-7b
mistralai/mistral-large
google/recurrentgemma-2b
mistralai/mistral-7b-instruct-v0.2
google/gemma-2-9b-it
nvidia/nemotron-4-340b-instruct
nvi

In [6]:
## Useful utility method for printing intermediate states
from langchain_core.runnables import RunnableLambda
from functools import partial

def RPrint(preface="State: "):
    def print_and_return(x, preface=""):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def PPrint(preface="State: "):
    def print_and_return(x, preface=""):
        pprint(preface, x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

----

<br>

## 1: Keeping Variables Flowing

In this example, lets try to implement the logic with chains by **creating**, **mutating**, and **consuming** states. These states were passed around as dictionaries with descriptive keys and useful values, and the values would be used to supply follow-up routines with the info they need to operate.

In [7]:
%%time
## ^^ This notebook is timed, which will print out how long it all took

from langchain_core.runnables import RunnableLambda
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from typing import List, Union
from operator import itemgetter

## Zero-shot classification prompt and chain w/ explicit few-shot prompting
sys_msg = (
    "Choose the most likely topic classification given the sentence as context."
    " Only one word, no explanation.\n[Options : {options}]"
)

zsc_prompt = ChatPromptTemplate.from_template(
    f"{sys_msg}\n\n"
    "[[The sea is awesome]][/INST]boat</s><s>[INST]"
    "[[{input}]]"
)

## Define simple instruct_model
instruct_chat = ChatNVIDIA(model="mistralai/mistral-7b-instruct-v0.2")
instruct_llm = instruct_chat | StrOutputParser()
one_word_llm = instruct_chat.bind(stop=[" ", "\n"]) | StrOutputParser()

zsc_chain = zsc_prompt | one_word_llm

## Function that just prints out the first word of the output. With early stopping bind
def zsc_call(input, options=["car", "boat", "airplane", "bike"]):
    return zsc_chain.invoke({"input" : input, "options" : options}).split()[0]

print("-" * 80)
print(zsc_call("Should I take the next exit, or keep going to the next one?"))

print("-" * 80)
print(zsc_call("I get seasick, so I think I'll pass on the trip"))

print("-" * 80)
print(zsc_call("I'm scared of heights, so flying probably isn't for me"))

--------------------------------------------------------------------------------
car
--------------------------------------------------------------------------------
boat
--------------------------------------------------------------------------------
air
CPU times: user 65 ms, sys: 7.9 ms, total: 72.9 ms
Wall time: 1.54 s


<br>

This chain makes several design decisions that make it very easy to use, key among them the following:

**We want it to act like a function, so all we want it to do is generate the output and return it.**

This makes the chain extremely natural for inclusion as a module in a larger chain system. For example, the following chain will take a string, extract the most likely topic, and then generate a new sentence based on the topic:



In [8]:
%%time
## ^^ This notebook is timed, which will print out how long it all took
gen_prompt = ChatPromptTemplate.from_template(
    "Make a new sentence about the the following topic: {topic}. Be creative!"
)

gen_chain = gen_prompt | instruct_llm

input_msg = "I get seasick, so I think I'll pass on the trip"
options = ["car", "boat", "airplane", "bike"]

chain = (
    ## -> {"input", "options"}
    {'topic' : zsc_chain}
    | PPrint()
    ## -> {**, "topic"}
    | gen_chain
    ## -> string
)

chain.invoke({"input" : input_msg, "options" : options})

State: 
{'topic': ' boat'}

CPU times: user 68.3 ms, sys: 4.04 ms, total: 72.3 ms
Wall time: 1.25 s


" As the sun began to set, the children's eyes gleamed with excitement as they rowed their makeshift paper boat through the sea of rippling bathwater in the living room, creating waves of laughter and magic."

<br>

However, it's a bit problematic when you want to keep the information flowing since we lose the topic and input variables in generating our response. If we wanted do something with both the output and the input, we'd need a to make sure that both variables pass through.

To do this, we can use the mapping runnable (i.e. interpretted from a dictionary or using manual `RunnableMap`) to pass both of the variables through by assigning the output of our chain to just a single key and letting the other keys propagate as desired. Alternatively, we could also use `RunnableAssign` to merge the state-consuming chain's output with the input dictionary by default.

With this technique, we can propagate whatever we want through our chain system:

In [10]:
%%time
## ^^ This notebook is timed, which will print out how long it all took

from langchain.schema.runnable import RunnableBranch, RunnablePassthrough
from langchain.schema.runnable.passthrough import RunnableAssign
from functools import partial

big_chain = (
    PPrint()
    ## Manual mapping. Can be useful sometimes and inside branch chains
    | {'input' : lambda d: d.get('input'), 'topic' : zsc_chain}
    | PPrint()
    ## RunnableAssign passing. Better for running state chains by default
    | RunnableAssign({'generated' : gen_chain})
    | PPrint()
    ## Using the input and generation together
    | RunnableAssign({'combination' : (
        ChatPromptTemplate.from_template(
            "Consider the following passages:"
            "\nP1: {input}"
            "\nP2: {generated}"
            "\n\nCombine the ideas from both sentences into one simple one."
        )
        | instruct_llm
    )})
    | PPrint()
)

output = big_chain.invoke({
    "input" : "I get seasick, so I think I'll pass on the trip",
    "options" : ["car", "boat", "airplane", "bike", "unknown"]
})
pprint("Final Output: ", output)

State: 
{
    'input': "I get seasick, so I think I'll pass on the trip",
    'options': ['car', 'boat', 'airplane', 'bike', 'unknown']
}

State: 
{'input': "I get seasick, so I think I'll pass on the trip", 'topic': ' boat'}

State: 
{
    'input': "I get seasick, so I think I'll pass on the trip",
    'topic': ' boat',
    'generated': " As the sun began to set, the children's eyes gleamed with excitement as they rowed their 
makeshift paper boat through the sea of rippling bathwater in the living room, creating waves of laughter and 
magic."
}

State: 
{
    'input': "I get seasick, so I think I'll pass on the trip",
    'topic': ' boat',
    'generated': " As the sun began to set, the children's eyes gleamed with excitement as they rowed their 
makeshift paper boat through the sea of rippling bathwater in the living room, creating waves of laughter and 
magic.",
    'combination': " I enjoy watching children's excitement as they play with paper boats in bathwater, but I get 
seasick myself so won't join them."
}

Final Output: 
{
    'input': "I get seasick, so I think I'll pass on the trip",
    'topic': ' boat',
    'generated': " As the sun began to set, the children's eyes gleamed with excitement as they rowed their 
makeshift paper boat through the sea of rippling bathwater in the living room, creating waves of laughter and 
magic.",
    'combination': " I enjoy watching children's excitement as they play with paper boats in bathwater, but I get 
seasick myself so won't join them."
}

CPU times: user 149 ms, sys: 3.28 ms, total: 152 ms
Wall time: 2.04 s


<br>

#### **Runnable Extraction Module**

We can use this Runnable that wraps the functionality of the Pydantic class and streamlines the prompting, generating, and updating of the knowledge base:

In [12]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import Dict, Union, Optional
from langchain.output_parsers import PydanticOutputParser

In [13]:
################################################################################
## Definition of RExtract
def RExtract(pydantic_class, llm, prompt):
    '''
    Runnable Extraction module
    Returns a knowledge dictionary populated by slot-filling extraction
    '''
    parser = PydanticOutputParser(pydantic_object=pydantic_class)
    instruct_merge = RunnableAssign({'format_instructions' : lambda x: parser.get_format_instructions()})
    def preparse(string):
        if '{' not in string: string = '{' + string
        if '}' not in string: string = string + '}'
        string = (string
            .replace("\\_", "_")
            .replace("\n", " ")
            .replace("\]", "]")
            .replace("\[", "[")
        )
        # print(string)  ## Good for diagnostics
        return string
    return instruct_merge | prompt | llm | preparse | parser

<br>

#### **Dynamic Knowledge Base Updates**

Now, let's create a system that continually updates the Knowledge Base throughout the conversation. This is done by feeding the current state of the Knowledge Base, along with new user inputs, back into the system for ongoing updates.

In [14]:
class KnowledgeBase(BaseModel):
    firstname: str = Field('unknown', description="Chatting user's first name, unknown if unknown")
    lastname: str = Field('unknown', description="Chatting user's last name, unknown if unknown")
    location: str = Field('unknown', description="Where the user is located")
    summary: str = Field('unknown', description="Running summary of conversation. Update this with new input")
    response: str = Field('unknown', description="An ideal response to the user based on their new message")


parser_prompt = ChatPromptTemplate.from_template(
    "You are chatting with a user. The user just responded ('input'). Please update the knowledge base."
    " Record your response in the 'response' tag to continue the conversation."
    " Do not hallucinate any details, and make sure the knowledge base is not redundant."
    " Update the entries frequently to adapt to the conversation flow."
    "\n{format_instructions}"
    "\n\nOLD KNOWLEDGE BASE: {know_base}"
    "\n\nNEW MESSAGE: {input}"
    "\n\nNEW KNOWLEDGE BASE:"
)

## Switch to a more powerful base model
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1") | StrOutputParser()

extractor = RExtract(KnowledgeBase, instruct_llm, parser_prompt)
info_update = RunnableAssign({'know_base' : extractor})

## Initialize the knowledge base and see what you get
state = {'know_base' : KnowledgeBase()}
state['input'] = "My name is Carmen Sandiego! Guess where I am! Hint: It's somewhere in the United States."
state = info_update.invoke(state)
pprint(state)

{
    'know_base': KnowledgeBase(
        firstname='Carmen',
        lastname='Sandiego',
        location='unknown',
        summary='The user has introduced herself as Carmen Sandiego and asked to guess her location. The location 
is somewhere in the United States.',
        response='Hello Carmen Sandiego! That sounds like an exciting game. Is there any other hint you can give me
about your location?'
    ),
    'input': "My name is Carmen Sandiego! Guess where I am! Hint: It's somewhere in the United States."
}

In [15]:
state['input'] = "I'm in a place considered the birthplace of Jazz."
state = info_update.invoke(state)
pprint(state)

{
    'know_base': KnowledgeBase(
        firstname='Carmen',
        lastname='Sandiego',
        location='unknown',
        summary='The user, who has introduced herself as Carmen Sandiego, is playing a game where she provides 
hints about her location and the chatbot has to guess. The initial location given was somewhere in the United 
States. She has now provided a new hint that she is in a place considered the birthplace of Jazz.',
        response="The birthplace of Jazz... That's a very interesting hint! Based on that, I would guess that you 
are in New Orleans, Louisiana. Am I getting closer?"
    ),
    'input': "I'm in a place considered the birthplace of Jazz."
}

In [16]:
state['input'] = "Yeah, I'm in New Orleans... How did you know?"
state = info_update.invoke(state)
pprint(state)

{
    'know_base': KnowledgeBase(
        firstname='Carmen',
        lastname='Sandiego',
        location='New Orleans, Louisiana',
        summary='The user, Carmen Sandiego, was playing a game where she provided hints about her location. She 
initially gave a hint that she was somewhere in the United States, and later revealed that she was in a place 
considered the birthplace of Jazz. The chatbot correctly guessed that she was in New Orleans, Louisiana.',
        response="I'm glad I could guess it correctly! I used the hint you gave about being in the birthplace of 
Jazz, which led me to guess New Orleans, Louisiana."
    ),
    'input': "Yeah, I'm in New Orleans... How did you know?"
}

<br>

This example shows how a running state chain can be effectively utilized to manage a conversation with evolving context and requirements, making it a powerful tool for developing sophisticated interactive systems.

The next sections of this notebook will expand on these concepts by exploring two specific applications: **Document Knowledge Bases** and **Database-Querying Chatbots**.

----

<br>

## 2: Customer Service Bot


Let's create a simple database-like interface to get some customer information from a dictionary!

In [19]:
#######################################################################################
## Function that can be queried for information. Implementation details not important
def get_flight_info(d: dict) -> str:
    """
    Example of a retrieval function which takes a dictionary as key. Resembles SQL DB Query
    """
    req_keys = ['first_name', 'last_name', 'user_id']
    assert all((key in d) for key in req_keys), f"Expected dictionary with keys {req_keys}, got {d}"

    ## Static dataset. get_key and get_val can be used to work with it, and db is your variable
    keys = req_keys + ["borough", "plastic_waste", "metal_waste", "date", "property_address"]
    values = [
        ["Jane", "Doe", 12345, "Queens", "234 Lb", "645 Lb", "July 12th", "34 W, Lexington Ave"],
        ["John", "Smith", 54321, "Manhattan", "23 Lb", "23 Lb", "Aug 15th", "64, Bayonne St"],
        ["Alice", "Johnson", 98765, "Brooklyn", "34 Lb", "897 LbM", "Aug 2nd", "22, Jefferson Avenue"],
        ["Bob", "Brown", 56789, "Bronx", "75 Lb", "12 Lb", "June 4th", "204 E, Washington St"],
    ]
    get_key = lambda d: "|".join([d['first_name'], d['last_name'], str(d['user_id'])])
    get_val = lambda l: {k:v for k,v in zip(keys, l)}
    db = {get_key(get_val(entry)) : get_val(entry) for entry in values}

    # Search for the matching entry
    data = db.get(get_key(d))
    if not data:
        return (
            f"Based on {req_keys} = {get_key(d)}) from your knowledge base, no info on the user data was found."
            " This process happens every time new info is learned. If it's important, ask them to confirm this info."
        )
    return (
        f"For the client named {data['first_name']} {data['last_name']} the garbage collected on {data['date']} is {data['plastic_waste']} plastic and {data['metal_waste']} metal "
        f"at the building {data['property_address']}, which is located in {data['borough']}."
    )

#######################################################################################
## Usage example. Actually important

print(get_flight_info({"first_name" : "Jane", "last_name" : "Doe", "user_id" : 12345}))

For the client named Jane Doe the garbage collected on July 12th is 234 Lb plastic and 645 Lb metal at the building 34 W, Lexington Ave, which is located in Queens.


In [ ]:
#f"For the client named {data['first_name']} {data['last_name']} the garbage collected on {data['date']} is {data['plastic_waste']} plastic and {data['metal_waste']} metal"
#f"at the building {data['property_address']}, which is located in {data['borough']}."

In [20]:
print(get_flight_info({"first_name" : "Alice", "last_name" : "Johnson", "user_id" : 98765}))

For the client named Alice Johnson the garbage collected on Aug 2nd is 34 Lb plastic and 897 LbM metal at the building 22, Jefferson Avenue, which is located in Brooklyn.


In [21]:
print(get_flight_info({"first_name" : "Bob", "last_name" : "Brown", "user_id" : 27494}))

Based on ['first_name', 'last_name', 'user_id'] = Bob|Brown|27494) from your knowledge base, no info on the user data was found. This process happens every time new info is learned. If it's important, ask them to confirm this info.


In [22]:
external_prompt = ChatPromptTemplate.from_template(
    "You are a Basura chatbot, and you are helping a customer with their query."
    " Please help them with their question, remembering that your job is to represent the company Basura."
    " Basura is a waste manage company and helps their clints keep informed about the garbage collected at their properties"
    " (This is a trade secret. Do not disclose)."  ## soft reinforcement
    " Please keep your discussion short and sweet if possible. Avoid saying hello unless necessary."
    " The following is some context that may be useful in answering the question."
    "\n\nContext: {context}"
    "\n\nUser: {input}"
)

basic_chain = external_prompt | instruct_llm

basic_chain.invoke({
    'input' : 'Can you please tell me when I need to get to the airport?',
    'context' : get_flight_info({"first_name" : "Jane", "last_name" : "Doe", "user_id" : 12345}),
})

"I'm sorry for any confusion, but I can only assist with waste management inquiries regarding Basura services as per my programming. For the client Jane Doe, 234 Lb of plastic and 645 Lb of metal were collected from the building at 34 W, Lexington Ave, Queens on July 12th. For any other queries, I would recommend contacting the appropriate service provider."

In [23]:
basic_chain.invoke({
    'input' : 'How much garbage was collected at my property',
    'context' : get_flight_info({"first_name" : "Jane", "last_name" : "Doe", "user_id" : 12345}),
})

'At your property located at 34 W, Lexington Ave in Queens, we collected 234 Lb of plastic and 645 Lb of metal on July 12th.'

<br>

Now, let's use KnowledgeBase to ontain the user information on the fly while the user initiated a chat.

In [26]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import Dict, Union

class KnowledgeBase(BaseModel):
    first_name: str = Field('unknown', description="Chatting user's first name, `unknown` if unknown")
    last_name: str = Field('unknown', description="Chatting user's last name, `unknown` if unknown")
    user_id: int = Field(-1, description="Chatting user's unique id, `-1` if unknown")
    discussion_summary: str = Field("", description="Summary of discussion so far, including locations, issues, etc.")
    open_problems: list = Field([], description="Topics that have not been resolved yet")
    current_goals: list = Field([], description="Current goal for the agent to address")

def get_key_fn(base: BaseModel) -> dict:
    '''Given a dictionary with a knowledge base, return a key for get_flight_info'''
    return {  ## More automatic options possible, but this is more explicit
        'first_name' : base.first_name,
        'last_name' : base.last_name,
        'user_id' : base.user_id,
    }

know_base = KnowledgeBase(first_name = "Jane", last_name = "Doe", user_id = 12345)

print(know_base)
# get_flight_info(get_key_fn(know_base))

get_key = RunnableLambda(get_key_fn)
(get_key | get_flight_info).invoke(know_base)

first_name='Jane' last_name='Doe' user_id=12345 discussion_summary='' open_problems=[] current_goals=[]


'For the client named Jane Doe the garbage collected on July 12th is 234 Lb plastic and 645 Lb metal at the building 34 W, Lexington Ave, which is located in Queens.'

In [28]:
for model in ChatNVIDIA.get_available_models():
  if "mistral" in model.id or "meta" in model.id:
    print(model.id)


meta/llama3-8b-instruct
meta/llama-3.1-405b-instruct
meta/llama3-70b-instruct
mistralai/codestral-22b-instruct-v0.1
liuhaotian/llava-v1.6-mistral-7b
meta/llama-3.1-8b-instruct
mistralai/mistral-7b-instruct-v0.3
mistralai/mistral-large
mistralai/mistral-7b-instruct-v0.2
meta/llama-3.1-70b-instruct
meta/codellama-70b
mistralai/mixtral-8x22b-instruct-v0.1
nv-mistralai/mistral-nemo-12b-instruct
meta/llama2-70b
mistralai/mamba-codestral-7b-v0.1
mistralai/mixtral-8x7b-instruct-v0.1


In [36]:
from langchain.schema.runnable import (
    RunnableBranch,
    RunnableLambda,
    RunnableMap,       ## Wrap an implicit "dictionary" runnable
    RunnablePassthrough,
)
from langchain.schema.runnable.passthrough import RunnableAssign

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, SystemMessage, ChatMessage, AIMessage
from typing import Iterable
import gradio as gr

external_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a chatbot for Basura - a waste management company in New York, and you are helping a client with their query."
        "Basura helps their clints keep informed about the garbage collected at their properties"
        "Note that Basura just provides data information and doesn't buy or handle garbage in any way"
        " Please chat with them! Stay concise and clear!"
        " Your running knowledge base is: {know_base}."
        " This is for you only; Do not mention it!"
        " \nUsing that, we retrieved the following: {context}\n"
        " If they provide info and the retrieval fails, ask to confirm their first/last name and user id."
        " Do not ask them any other personal info."
        " If it's not important to know about their property address or borough, do not ask."
        " The garbage data is updated manually in the datatbase by Basura's employees daily."
    )),
    ("assistant", "{output}"),
    ("user", "{input}"),
])

##########################################################################
## Knowledge Base Things

class KnowledgeBase(BaseModel):
    first_name: str = Field('unknown', description="Chatting user's first name, `unknown` if unknown")
    last_name: str = Field('unknown', description="Chatting user's last name, `unknown` if unknown")
    user_id: Optional[int] = Field(None, description="Chatting user's unique id, `-1` if unknown")
    discussion_summary: str = Field("", description="Summary of discussion so far, including locations, issues, etc.")
    open_problems: str = Field("", description="Topics that have not been resolved yet")
    current_goals: str = Field("", description="Current goal for the agent to address")

parser_prompt = ChatPromptTemplate.from_template(
    "You are a chat assistant representing Basura - a waste management company in New York, and are trying to track info about the conversation."
    " You have just recieved a message from the user. Please fill in the schema based on the chat."
    "\n\n{format_instructions}"
    "\n\nOLD KNOWLEDGE BASE: {know_base}"
    "\n\nASSISTANT RESPONSE: {output}"
    "\n\nUSER MESSAGE: {input}"
    "\n\nNEW KNOWLEDGE BASE: "
)

## Switch to a more powerful base model
chat_llm = ChatNVIDIA(model="meta/llama3-70b-instruct") | StrOutputParser()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1") | StrOutputParser()

external_chain = external_prompt | chat_llm


knowbase_getter = RExtract(KnowledgeBase, instruct_llm, parser_prompt)
database_getter = itemgetter('know_base') | get_key | get_flight_info

## These components integrate to make your internal chain
internal_chain = (
    RunnableAssign({'know_base' : knowbase_getter})
    | RunnableAssign({'context' : database_getter})
)


state = {'know_base' : KnowledgeBase()}

def chat_gen(message, history=[], return_buffer=True):

    ## Pulling in, updating, and printing the state
    global state
    state['input'] = message
    state['history'] = history
    state['output'] = "" if not history else history[-1][1]

    ## Generating the new state from the internal chain
    state = internal_chain.invoke(state)
    #print("State after chain run:")
    #pprint({k:v for k,v in state.items() if k != "history"})

    ## Streaming the results
    buffer = ""
    for token in external_chain.stream(state):
        buffer += token
        yield buffer if return_buffer else token

In [37]:
state = {'know_base' : KnowledgeBase()}

chatbot = gr.Chatbot(value=[[None, "Hello! I'm your Basura agent! How can I help you?"]])
demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue().launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://5e5373292c38c70fff.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


{
    'know_base': KnowledgeBase(
        first_name='unknown',
        last_name='unknown',
        user_id=-1,
        discussion_summary='User needs information about their data.',
        open_problems="User's data information requirement.",
        current_goals='Provide user with requested information about their data.'
    ),
    'input': 'i need info on my data',
    'output': "Hello! I'm your Basura agent! How can I help you?",
    'context': "Based on ['first_name', 'last_name', 'user_id'] = unknown|unknown|-1) from your knowledge base, no 
info on the user data was found. This process happens every time new info is learned. If it's important, ask them 
to confirm this info."
}

{
    'know_base': KnowledgeBase(
        first_name='Jane',
        last_name='Doe',
        user_id=12345,
        discussion_summary='User needs information about their data. User confirmed their first name is Jane, last 
name is Doe, and user ID is 12345.',
        open_problems="User's data information requirement.",
        current_goals='Provide user with requested information about their data.'
    ),
    'input': 'Jane doe and 12345',
    'output': "I'd be happy to help you with that. To provide you with the correct information, could you please 
confirm your first name, last name, and user ID? This will help me retrieve your data from our system.",
    'context': 'For the client named Jane Doe the garbage collected on July 12th is 234 Lb plastic and 645 Lb metal
at the building 34 W, Lexington Ave, which is located in Queens.'
}

{
    'know_base': KnowledgeBase(
        first_name='Jane',
        last_name='Doe',
        user_id=12345,
        discussion_summary='User needs information about their data. User confirmed their first name is Jane, last 
name is Doe, and user ID is 12345. User requested information about their garbage collection data at their property
in Brooklyn.',
        open_problems="User's data information requirement about their Brooklyn property.",
        current_goals='Provide user with requested information about their data at Brooklyn property.'
    ),
    'input': 'Yes, do you have data on my property at Brooklyn',
    'output': "Hello Jane Doe! I've retrieved your data. According to our records, on July 12th, 234 Lb of plastic 
and 645 Lb of metal were collected from your building located at 34 W, Lexington Ave in Queens. Is there anything 
else you'd like to know about your garbage collection data?",
    'context': 'For the client named Jane Doe the garbage collected on July 12th is 234 Lb plastic and 645 Lb metal
at the building 34 W, Lexington Ave, which is located in Queens.'
}

{
    'know_base': KnowledgeBase(
        first_name='Jane',
        last_name='Doe',
        user_id=12345,
        discussion_summary='User needs information about their data. User confirmed their first name is Jane, last 
name is Doe, and user ID is 12345. User requested information about their garbage collection data at their property
in Brooklyn.',
        open_problems="User's data information requirement about their Brooklyn property.",
        current_goals='Provide user with requested information about their data at Brooklyn property.'
    ),
    'input': 'Its Jane Doe and 12345',
    'output': 'I apologize for the mistake earlier. I understand you are looking for data on your property in 
Brooklyn, not Queens. Can you please confirm your first name, last name, and user ID to ensure I retrieve the 
correct information for you?',
    'context': 'For the client named Jane Doe the garbage collected on July 12th is 234 Lb plastic and 645 Lb metal
at the building 34 W, Lexington Ave, which is located in Queens.'
}

{
    'know_base': KnowledgeBase(
        first_name='Jane',
        last_name='Doe',
        user_id=12345,
        discussion_summary='User needs information about their data. User confirmed their first name is Jane, last 
name is Doe, and user ID is 12345. User requested information about their garbage collection data at their 
properties in Brooklyn and Queens. User confirmed having properties in both locations.',
        open_problems="User's data information requirement about their Brooklyn and Queens properties.",
        current_goals='Provide user with requested information about their data at Brooklyn and Queens properties.'
    ),
    'input': 'Yes i have properties both at Brooklyn and Queens',
    'output': "Thank you for confirming, Jane! I've checked our database again. It seems that I couldn't find any 
data associated with your Brooklyn property. I apologize for the mistake earlier. The data I retrieved earlier was 
for a different location. Can you please give me a moment to investigate further? Additionally, could you please 
confirm that your property is indeed located in Brooklyn?",
    'context': 'For the client named Jane Doe the garbage collected on July 12th is 234 Lb plastic and 645 Lb metal
at the building 34 W, Lexington Ave, which is located in Queens.'
}

{
    'know_base': KnowledgeBase(
        first_name='Jane',
        last_name='Doe',
        user_id=12345,
        discussion_summary='User needs information about their data. User confirmed their first name is Jane, last 
name is Doe, and user ID is 12345. User requested information about their garbage collection data at their 
properties in Brooklyn and Queens. User confirmed having properties in both locations. For the Queens property, 234
Lb of plastic and 645 Lb of metal were collected on July 12th at the building located at 34 W, Lexington Ave. 
Further investigation about the Brooklyn property was requested.',
        open_problems="User's data information requirement about their Brooklyn and Queens properties.",
        current_goals='Provide user with requested information about their Brooklyn property data.'
    ),
    'input': 'yes please',
    'output': "I've re-checked our database. For your Queens property, I found that on July 12th, we recorded 234 
Lb of plastic and 645 Lb of metal collected at the building located at 34 W, Lexington Ave. Would you like me to 
investigate further about your Brooklyn property?",
    'context': 'For the client named Jane Doe the garbage collected on July 12th is 234 Lb plastic and 645 Lb metal
at the building 34 W, Lexington Ave, which is located in Queens.'
}

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5e5373292c38c70fff.gradio.live
